# Enhancing Cache Effectiveness

In this lab, you’ll learn several techniques to make your cache more accurate—like threshold tuning, cross-encoders, LLM checks, and fuzzy matching.


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Setup Environment

In [2]:
%load_ext autoreload
%autoreload 2
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from cache.cross_encoder import CrossEncoder
from cache.faq_data_container import FAQDataContainer
from cache.llm_evaluator import LLMEvaluator
from cache.wrapper import SemanticCacheWrapper
from cache.evals import CacheEvaluator
from cache.config import config

# Set up plotting style
plt.style.use("default")
sns.set_palette("husl")

print("📦 Libraries and evaluation utilities imported successfully")

📦 Libraries and evaluation utilities imported successfully


## Load data and setup cache

In [3]:
cache_wrapper = SemanticCacheWrapper.from_config(config)

✅ Redis is running and accessible!


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: redis/langcache-embed-v1
INFO:redisvl.index.index:Index already exists, not overwriting.


In [4]:
data_container = FAQDataContainer()

test_queries = data_container.test_df["question"].tolist()

Loaded 8 FAQ entries
Loaded 80 test queries


In [5]:
cache_wrapper.hydrate_from_df(data_container.faq_df)
cache_results = cache_wrapper.check_many(
    test_queries, distance_threshold=0.3
)

evaluator = CacheEvaluator(
    true_labels=data_container.label_cache_hits(cache_results),
    cache_results=cache_results,
)
evaluator.report_metrics()

Column
    [0] Markdown(str)
    [1] Row
        [0] DataFrame(DataFrame, width=200)
        [1] DataFrame(Styler)

## Threshold sweep

In [6]:
cache_results = cache_wrapper.check_many(
    test_queries, distance_threshold=1
)
evaluator = CacheEvaluator.from_full_retrieval(
    true_labels=data_container.label_cache_hits(cache_results),
    cache_results=cache_results,
)

evaluator.report_threshold_sweep(
    metric_to_maximize="f1_score",
    metrics_to_plot=["f1_score", "precision", "recall"],
)

Row
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] DataFrame(DataFrame, width=200)
            [1] DataFrame(Styler)
    [1] Matplotlib(Figure, format='svg', height=420, tight=True)

## Cross Encoder Reranking

In [7]:
# Initialize cross encoder for reranking
cross_encoder = CrossEncoder("Alibaba-NLP/gte-reranker-modernbert-base")

# Register cross encoder as a reranker with the cache wrapper
cache_wrapper.register_reranker(cross_encoder.create_reranker())

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [8]:
# Test cross encoder directly (for demonstration)
cross_encoder.predict(
    [
        "what is the capital of China?",
        "how to implement quick sort in python?",
        "how to implement quick sort in python?",
    ],
    ["Beijing", "Introduction of quick sort", "The weather is nice today"],
)

[0.8944211602210999, 0.9210741519927979, 0.15843459963798523]

In [9]:
sentences = [
    "The bank raised its interest rates.",
    "The river overflowed near the bank after heavy rain."
]

langcache_distance = cache_wrapper.pair_distance(
    sentences[0], sentences[1]
)
cross_encoder_distance = cross_encoder.pair_distance(
    sentences[0], sentences[1]
)

langcache_distance, cross_encoder_distance

(0.46540400063924103, 0.6081287264823914)

In [10]:
cache_results = cache_wrapper.check_many(
    test_queries,
    distance_threshold=1,
    num_results=10,
    use_reranker_distance=True,
)
evaluator = CacheEvaluator.from_full_retrieval(
    true_labels=data_container.label_cache_hits(cache_results),
    cache_results=cache_results,
)

evaluator.report_threshold_sweep()

Row
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] DataFrame(DataFrame, width=200)
            [1] DataFrame(Styler)
    [1] Matplotlib(Figure, format='svg', height=420, tight=True)

## LLM Reranking in a Tiered System

In [11]:
from cache.config import load_openai_key

load_openai_key()

> OpenAI API key is already loaded in the environment


In [12]:
llm = LLMEvaluator.construct_with_ollama()

cache_wrapper.clear_reranker()
cache_wrapper.register_reranker(llm.create_reranker(batch_size=4))

In [13]:
cache_results = cache_wrapper.check_many(
    test_queries,
    distance_threshold=0.2828,
    num_results=1,
    show_progress=True,
)

100%|██████████| 80/80 [00:50<00:00,  1.57it/s]


In [14]:
evaluator = CacheEvaluator.from_full_retrieval(
    true_labels=data_container.label_cache_hits(cache_results),
    cache_results=cache_results,
)

evaluator.report_metrics()

Column
    [0] Markdown(str)
    [1] Row
        [0] DataFrame(DataFrame, width=200)
        [1] DataFrame(Styler)

## Fuzzy Matching

In [15]:
def fuzzify_string(str, its=3):
    for i in range(its):
        str_list = list(str)
        i = np.random.randint(0, len(str) - 1)
        str_list[i], str_list[i + 1] = str_list[i + 1], str_list[i]
        str = "".join(str_list)
    return str

In [16]:
fuzzy_queries = []
valid_query_map = set()
for q in data_container.faq_df["question"].tolist():
    for dificulty in [2, 3, 4, 10, 10000]:
        new_entry = fuzzify_string(q, dificulty)
        fuzzy_queries.append(new_entry)
        valid_query_map.add((new_entry, q))

In [17]:
fuzzy_queries[:5]

['How od I get a erfund?',
 'oHw do  Iegt a refund?',
 'Hwo  doI gte a refund?',
 'H owo dI ge ta renfdu?',
 'dwda og u Hrfe I ?eton']

In [18]:
from cache.fuzzy_cache import FuzzyCache

fuzzy_cache = FuzzyCache()
fuzzy_cache.hydrate_from_df(data_container.faq_df)
fuzzy_retrievals = fuzzy_cache.check_many(fuzzy_queries)

In [19]:
fuzzy_retrievals[:5]

[(Query: 'How od I get a erfund?', Matches: ['How do I get a refund?']),
 (Query: 'oHw do  Iegt a refund?', Matches: ['How do I get a refund?']),
 (Query: 'Hwo  doI gte a refund?', Matches: ['How do I get a refund?']),
 (Query: 'H owo dI ge ta renfdu?', Matches: ['How do I get a refund?']),
 (Query: 'dwda og u Hrfe I ?eton', Matches: ['How do I get a refund?'])]

In [20]:
valid_fuzzy_labeling = [
    (r.query, r.matches[0].prompt) in valid_query_map if len(r.matches) > 0 else False
    for r in fuzzy_retrievals
]

In [21]:
CacheEvaluator.from_full_retrieval(
    true_labels=valid_fuzzy_labeling,
    cache_results=fuzzy_retrievals,
).report_metrics(distance_threshold=0.6)

Column
    [0] Markdown(str)
    [1] Row
        [0] DataFrame(DataFrame, width=200)
        [1] DataFrame(Styler)

In [22]:
cache_wrapper.cache.clear()